## Supuestos estructurales del Modelo Clásico de Regresión Lineal (MCRL)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns # Visualización de datos
import statsmodels.stats.api as sms

In [2]:
data=pd.read_csv("_____________________________", delimiter="\t")

In [1]:
sns.regplot(x=________, y=________);

NameError: name 'sns' is not defined

**Supuesto #6:**

In [7]:
#Observaciones mayores a parametros

In [9]:
Modelo_de_Prueba=sm.OLS(data["CM"]_____,sm.add_constant(__________________________))

**Supuesto #7:**

In [4]:
#Variabilidad en las variables

In [10]:
sm.add_constant(data).apply(_______)

NameError: name '_______' is not defined

In [11]:
pd.DataFrame(data.apply(________),columns=["Desviación estándar"])

NameError: name '________' is not defined

**Supuesto #8:**

In [15]:
#Multicolinealidad

In [12]:
correlaciones=__________
fig,ax=plt.subplots()
sns.heatmap(_________)
plt.show()

NameError: name '__________' is not defined

In [13]:
correlaciones=___________
fig,ax=plt.subplots()
sns.heatmap(___________,cmap="RdYlGn")
plt.show()

NameError: name '___________' is not defined

In [14]:
correlaciones2=___________
fig,ax=plt.subplots()
fig.suptitle("Correlaciones CM-PGNP",fontsize=____,color=_______)
sns.heatmap(___________________________)
plt.show()

NameError: name '___________' is not defined

In [16]:
# Vif 

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["Nombre"] = __________
  
# Calculo de VIF
vif_data["VIF"] = [variance_inflation_factor(_________, i)
                          for i in range(____________)]
print(vif_data)

NameError: name '__________' is not defined

**Supuesto de no sesgo de especificación**

In [20]:
Modelo1=sm.OLS(data["CM"],sm.add_constant(data[["PGNP","FLR"]]))

In [21]:
Resultados1=Modelo1.fit()

In [22]:
Valores_estimados=___________________

NameError: name '___________________' is not defined

In [23]:
Residuos=__________________

NameError: name '__________________' is not defined

In [24]:
# Prueba CUSUM para medir cambio estructural

# H0 : No hay cambio estructural
# H1: Hay cambio estructural

ResultadosCUSUM = sms.________________________
print(ResultadosCUSUM)

AttributeError: module 'statsmodels.stats.api' has no attribute '________________________'

In [25]:
# Para evaluar si la forma funcional es la correcta se utiliza el test de RESET de Ramsey

ResultadosRESET = sms.___________(Resultados1)
Estadistico = _______________
p_valor = ________________

Nombres = ["Estadistico","p-valor"]
pd.DataFrame([___________,___________], index = Nombres, columns = ["Reset"])

# Como p-valor es mayor a 0.05 la regresión tiene una correcta forma funcional 

AttributeError: module 'statsmodels.stats.api' has no attribute '___________'

## Ejercicio

Realice un modelo de regresión lineal con la base de datos Ingreso_Ahorro disponible en la sesión 3 del moodle; adicionalmente, verifique el cumplimieneto de los supuesto vistos en clase.

Este archivo de datos contiene información sobre 26 automóviles, a saber, su marca, precio, millas por galón, calificación de reparación (en 1978), peso en libras, longitud en pulgadas y si el automóvil era extranjero o nacional. Aquí está el archivo de datos.

In [91]:
dataEjercicio=pd.read_csv("_____________________")

In [26]:
sns.catplot(______________)

NameError: name '______________' is not defined

In [27]:
sns.catplot(_____________, hue="foreign", kind="swarm", _________)

SyntaxError: positional argument follows keyword argument (<ipython-input-27-392ab2570258>, line 1)